In [2]:
import pandas as pd
import numpy as np
# df = pd.read_csv('data/cleaned_data.csv')

df = pd.read_csv('cleaned_data.csv')
    
# Add age groups
df['age_group'] = df['Age'].apply(lambda x: 
    'Gen Z' if x < 25 else 
    'Millennials' if x < 40 else 
    'Gen X' if x < 55 else 'Boomers'
)

# Add tenure groups
df['tenure_group'] = df['Tenure'].apply(lambda x:
    'New (0-2 years)' if x <= 2 else
    'Mid (3-7 years)' if x <= 7 else
    'Senior (8+ years)'
)

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

#ML dataset
ml_data = df.copy()

#ignore non-pred columns
non_pred_col = ['BenefitID', 'LastUsedDate', 'Comments', 'Month']
ml_data = ml_data.drop([col for col in non_pred_col if col in ml_data.columns], axis=1)

#encoding
cat_cols = ['Gender', 'Department', 'age_group', 'tenure_group', 'BenefitType', 'BenefitSubType']
label_encoders = {}

for col in cat_cols:
    if col in ml_data.columns:
        le = LabelEncoder()
        ml_data[f'{col}_encoded'] = le.fit_transform(ml_data[col].astype(str))
        label_encoders[col] = le
        ml_data = ml_data.drop(col, axis=1)

other_cols = ml_data.select_dtypes(exclude=[np.number]).columns
for col in other_cols:
    if col in ml_data.columns:
        # For non-numeric columns, either encode them or drop them
        if ml_data[col].dtype == 'object':
            le = LabelEncoder()
            ml_data[f'{col}_encoded'] = le.fit_transform(ml_data[col].astype(str))
            ml_data = ml_data.drop(col, axis=1)


# Prepare feature matrix (exclude target variables and benefit subtype one-hot columns for now)
feature_cols = [col for col in ml_data.columns if not col.startswith(('subcat_', 'UsageFrequency', 'SatisfactionScore'))]
X = ml_data[feature_cols]

# Ensure all columns are numeric
X = X.select_dtypes(include=[np.number])

# Scale numerical features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

In [6]:
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score

cluster_features = ['age_group_encoded', 'tenure_group_encoded', 'Department_encoded']

available_features = [col for col in cluster_features if col in ml_data.columns]

cluster_data = ml_data[cluster_features].fillna(ml_data[cluster_features].median())

cluster_scaler = StandardScaler()
cluster_data_scaled = cluster_scaler.fit_transform(cluster_data)

# Determine optimal number of clusters using silhouette score
silhouette_scores = []
k_range = range(2, 8)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(cluster_data_scaled)
    silhouette_avg = silhouette_score(cluster_data_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)

# Find optimal k
optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_k}")

# Apply K-Means with optimal k
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42)
ml_data['employee_segment'] = kmeans_final.fit_predict(cluster_data_scaled)
df['employee_segment'] = ml_data['employee_segment']

# Analyze segments
segment_analysis = df.groupby('employee_segment').agg({
    'Age': 'mean',
    'Tenure': 'mean'
}).round(2)


#validation scores
davies_score = davies_bouldin_score(cluster_data_scaled, ml_data['employee_segment'])
sil_score = silhouette_score(cluster_data_scaled, ml_data['employee_segment'])

print(f"Silhouette Score: {sil_score:.4f}")
print(f"Davies-Bouldin Score: {davies_score:.4f}")

Optimal number of clusters: 6
Silhouette Score: 0.4378
Davies-Bouldin Score: 0.8468


In [11]:
segment_top = []
segment_bot = []
seg_rec = []

for clust_id in sorted(df['employee_segment'].unique()):
    clust_data = df[df['employee_segment'] == clust_id]
    clust_size = len(clust_data)

    print(f"\nCluster #: {clust_id} Cluster size:{clust_size}, {clust_size/len(df)*100:.1f}% of employees)")
    
    print(f"Avg Cluster Age: {clust_data['Age'].mean():.1f} years")
    print(f"Avg Cluster Tenure: {clust_data['Tenure'].mean():.1f} years")
    print(f"Avg Cluster Usage Frequency: {clust_data['UsageFrequency'].mean():.2f}")
    print(f"Avg Cluster Satisfaction: {clust_data['SatisfactionScore'].mean():.2f}/5.0")
    
    #top benefits by usage
    top_benefits = clust_data.groupby('BenefitSubType')['UsageFrequency'].mean().sort_values(ascending=False).head(3)
    print("\nTOP 3 (by usage):")
    top3 = []
    for i, (benefit, usage) in enumerate(top_benefits.items(), 1):
        top3.append(benefit)
    segment_top.append(top3)
    print(top3)
    
    #low usage benefits
    low_usage = clust_data.groupby('BenefitSubType')['UsageFrequency'].mean().sort_values().head(3)
    print(f"\nBOT 3(by usage):")
    bot3 = []
    for i, (benefit, usage) in enumerate(low_usage.items(), 1):
        bot3.append(benefit)
    segment_bot.append(bot3)
    print(bot3)

    high_usage_benefits = clust_data.groupby('BenefitSubType')['UsageFrequency'].mean()
    high_usage_benefits = high_usage_benefits[high_usage_benefits > high_usage_benefits.quantile(0.8)]
    
    #top 3 benefits if there is
    for benefit in high_usage_benefits.index[:3]:
        related_category = clust_data[clust_data['BenefitSubType'] == benefit]['BenefitType'].iloc[0]
        other_in_category = clust_data[
            (clust_data['BenefitType'] == related_category) & 
            (clust_data['BenefitSubType'] != benefit)
        ]['BenefitSubType'].unique()
        
    seg_rec.append(list(other_in_category)[:3])
    print('Cluster recs\n')
    print(list(other_in_category)[:3])



Cluster #: 0 Cluster size:1032, 13.5% of employees)
Avg Cluster Age: 28.5 years
Avg Cluster Tenure: 4.0 years
Avg Cluster Usage Frequency: 3.43
Avg Cluster Satisfaction: 2.96/5.0

TOP 3 (by usage):
['Monthly Communications', 'HMO Family', 'Dependent Coverage']

BOT 3(by usage):
['Healthcare FSA', 'After-School Care', 'On-Site Infant Care']
Cluster recs

['401k Standard Matching', '401k Basic Matching', '401k Investment Fees']

Cluster #: 1 Cluster size:976, 12.8% of employees)
Avg Cluster Age: 33.4 years
Avg Cluster Tenure: 4.2 years
Avg Cluster Usage Frequency: 3.22
Avg Cluster Satisfaction: 2.97/5.0

TOP 3 (by usage):
['401k High Contribution', 'Supplemental High Amount', 'Premium Discount Tier 1']

BOT 3(by usage):
['HDHP Individual', 'On-Site Infant Care', 'Undergraduate Degree']
Cluster recs

['Basic Coverage', 'Supplemental High Amount', 'Supplemental Standard']

Cluster #: 2 Cluster size:1567, 20.5% of employees)
Avg Cluster Age: 50.4 years
Avg Cluster Tenure: 19.2 years
Avg Cl

In [13]:
emp_id_get = ml_data.drop_duplicates(subset=['EmployeeID'], keep='first')

In [14]:
def suggest_benefits(employee_id, top_n=3):
    # Normalize type
    emp_rows = df[df['EmployeeID'].astype(str).str.strip() == str(employee_id).strip()]
    if emp_rows.empty:
        return []  

    emp_row = emp_rows.iloc[0]
    dept   = emp_row.get('Department')
    tenure = emp_row.get('tenure_group')
    age    = emp_row.get('age_group')

    if pd.isna(dept) or pd.isna(tenure) or pd.isna(age):
        return []

    peer_df = df[
        (df['Department']   == dept) &
        (df['tenure_group'] == tenure) &
        (df['age_group']    == age)
    ]
    if peer_df.empty:
        return []

    peer_benefits = peer_df.groupby('BenefitSubType')['UsageFrequency'].sum().sort_values(ascending=False)
    emp_benefits = set(emp_rows['BenefitSubType'].dropna())
    suggestions = [b for b in peer_benefits.index if b not in emp_benefits]

    return suggestions[:top_n]

In [15]:
segmentDF = df.drop_duplicates(subset=['EmployeeID'], keep='first')

In [16]:
for seg_id, tops in enumerate(segment_top):
    segmentDF.loc[segmentDF['employee_segment'] == seg_id, ['top1', 'top2', 'top3']] = tops

for seg_id, bots in enumerate(segment_bot):
    segmentDF.loc[segmentDF['employee_segment'] == seg_id, ['bot1', 'bot2', 'bot3']] = bots

for seg_id, segrecs in enumerate(segment_bot):
    segmentDF.loc[segmentDF['employee_segment'] == seg_id, ['seg_rec1', 'seg_rec2', 'seg_rec3']] = segrecs


for emp in segmentDF['EmployeeID']:
    recs = suggest_benefits(emp, top_n=3)
    segmentDF.loc[segmentDF['EmployeeID'] == emp, 'rec1'] = recs[0]
    segmentDF.loc[segmentDF['EmployeeID'] == emp, 'rec2'] = recs[1]
    segmentDF.loc[segmentDF['EmployeeID'] == emp, 'rec3'] = recs[2]

segmentDF = segmentDF.drop_duplicates(subset=['EmployeeID'], keep='first')
segmentDF['employee_segment'] = emp_id_get['employee_segment']


In [17]:
segmentDF.head(5)

,EmployeeID,BenefitID,UsageFrequency,LastUsedDate,Age,Gender,Department,Tenure,BenefitType,BenefitSubType,...,top3,bot1,bot2,bot3,seg_rec1,seg_rec2,seg_rec3,rec1,rec2,rec3
0,220,20,4,2024-05-03,64,Male,HR,35,Tuition Reimbursement,Undergraduate Degree,...,Supplemental High Amount,On-Site Infant Care,Premium Discount Tier 1,PPO Individual,On-Site Infant Care,Premium Discount Tier 1,PPO Individual,Healthcare FSA,HMO Family,Transit Subsidy
1,1820,26,1,2024-02-08,53,Male,Finance,2,Gym Membership,Family Membership,...,Supplemental High Amount,On-Site Infant Care,Premium Discount Tier 1,PPO Individual,On-Site Infant Care,Premium Discount Tier 1,PPO Individual,Healthcare FSA,Monthly Communications,Tier 1 Partners
2,285,16,2,2023-10-27,64,Male,Marketing,35,Health Insurance,HDHP Individual,...,401k Maximum Matching,After-School Care,Dependent Coverage,On-Site Infant Care,After-School Care,Dependent Coverage,On-Site Infant Care,Basic Coverage,Supplemental Standard,PPO Individual
3,4536,8,8,2024-07-03,32,Female,Sales,10,Wellness Programs,Premium Discount Tier 1,...,401k High Contribution,401k Catch-Up Contributions,Professional Certification,PPO Family,401k Catch-Up Contributions,Professional Certification,PPO Family,401k High Contribution,Supplemental High Amount,401k Catch-Up Contributions
4,1262,12,3,2024-04-13,42,Male,Finance,1,Tuition Reimbursement,Graduate Degree,...,Supplemental High Amount,On-Site Infant Care,Premium Discount Tier 1,PPO Individual,On-Site Infant Care,Premium Discount Tier 1,PPO Individual,Healthcare FSA,Monthly Communications,Tier 1 Partners


In [18]:
segmentDF.to_csv('data/segment_data.csv', index=False)